First, a directory will be created to store all the necessary files:

In [ ]:
!mkdir pgadmin_postgresql

It is necessary to execute (if not already done) the following commands:

```bash
sudo groupadd docker
sudo usermod -aG docker ${USER}
newgrp docker
```

These commands allow Docker to have sudo permissions, and later, it is not necessary to provide a password each time the docker-compose command is executed.

If necessary, with the `prune` and `remove-orphans` options, all orphaned nodes can be removed (if there are any, in my case, there were) and then delete all previously created and built ones:

In [132]:
!docker-compose down --remove-orphans
!docker builder prune --all --force

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Total reclaimed space: 0B


To run containers with Docker, it is necessary to first build and then "upping" the containers using the docker-compose command:

In [137]:
!docker-compose build
!docker-compose up -d

[+] Running 0/0
 ⠋ Network pgadmin_postgresql_default  Creating                            0.1s 
[+] Running 0/1
 ⠙ Network pgadmin_postgresql_default  Creating                            0.2s 
[+] Running 0/1
 ⠹ Network pgadmin_postgresql_default  Creating                            0.3s 
[+] Running 1/1
 ✔ Network pgadmin_postgresql_default  Created                             0.3s 
 ⠋ Container local_pgdb                Creating                            0.0s 
[+] Running 1/2
 ✔ Network pgadmin_postgresql_default  Created                             0.3s 
 ⠙ Container local_pgdb                Creating                            0.1s 
[+] Running 1/2
 ✔ Network pgadmin_postgresql_default  Created                             0.3s 
 ⠹ Container local_pgdb                Creating                            0.2s 
[+] Running 1/2
 ✔ Network pgadmin_postgresql_default  Created                             0.3s 
 ⠸ Container local_pgdb                Creating                            0.3

The containers being brought up are pgAdmin, a tool for managing the PostgreSQL database, and the PostgreSQL database service that will be used for data storage. In addition to the containers, the Dockerfile also includes the installation of Apache for using Apache Benchmark.

After successfully bringing up the service, with the `ps` option, we can see all running processes, and they are considered successfully started if their status indicates the last uptime:

In [22]:
!docker ps

CONTAINER ID   IMAGE            COMMAND                  CREATED          STATUS          PORTS                                            NAMES
3840d73158a3   dpage/pgadmin4   "/entrypoint.sh"         51 seconds ago   Up 45 seconds   443/tcp, 0.0.0.0:8888->80/tcp, :::8888->80/tcp   pgadmin4_container
ce2b940cbdf2   postgres         "docker-entrypoint.s…"   53 seconds ago   Up 47 seconds   0.0.0.0:5432->5432/tcp, :::5432->5432/tcp        local_pgdb


If the services are successfully started, it is possible to open pgAdmin at the link [localhost:8888](http://localhost:8888), where access can be made using the following credentials:

```py
# General section
server : "server"
# Connection section
hostname : "postgres"
port : "5432"
user : "pgsqladmin"
password : "pgsqladminpass"
```

**Note**: It is advisable to wait a bit for the complete startup of the services, as in some cases on my own computer, it took a bit more time for the page to become accessible.

After logging in, it is possible to perform a database restore located in the `pg_backup` directory to obtain the data that I personally used.

In addition to containers, it is necessary to install the psycopg2 package (a package for establishing a connection to a PostgreSQL database) and faker (a package for generating fake data) for Python:

In [7]:
!pip install psycopg2-binary
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.0 MB/s eta 0:00:00:00:0100:01


In [ ]:
# Generic exception to prevent the continuation of executing other commands after opening the notebook.
raise Exception("Stop right there!")

Similarly, in case the database restore is not performed, it is possible to recreate data for the 4 tables created with `db_scripts.py` and generate data using `fake_data.py`:


In [24]:
!python ./py_scripts/db_scripts.py

In [36]:
!python ./py_scripts/fake_data.py

Next, we need to import `cache_purge` function from `cache_purge.py`: 

In [46]:
from py_scripts.cache_purge import cache_purge

Next 2 lines of code are from `cache_purge` and are meant to purge cache by reloading cache, terminating backend, and there is a third option which needs to be executed in bash and it basically purges all cache locally on PC: 

In [134]:
#Reload cache
cache_purge(1)

Cache cleared successfully!


In [135]:
#Terminate backend purge cache
cache_purge(2)

Cache cleared successfully!


Command for purging all cache:

```sh
sudo sh -c "/usr/bin/echo 3 > /proc/sys/vm/drop_caches"
```

The following command is used to execute a benchmark on the database to determine changes before and after cache purge:

In [106]:
!ab -n 10000 -c 200 http://127.0.0.1:5432/

This is ApacheBench, Version 2.3 <$Revision: 1903618 $>
Copyright 1996 Adam Twiss, Zeus Technology Ltd, http://www.zeustech.net/
Licensed to The Apache Software Foundation, http://www.apache.org/

Benchmarking 127.0.0.1 (be patient)
Completed 1000 requests
Completed 2000 requests
Completed 3000 requests
Completed 4000 requests
Completed 5000 requests
Completed 6000 requests
Completed 7000 requests
Completed 8000 requests
Completed 9000 requests
Completed 10000 requests
Finished 10000 requests


Server Software:        
Server Hostname:        127.0.0.1
Server Port:            5432

Document Path:          /
Document Length:        0 bytes

Concurrency Level:      200
Time taken for tests:   28.166 seconds
Complete requests:      10000
Failed requests:        0
Total transferred:      0 bytes
HTML transferred:       0 bytes
Requests per second:    355.04 [#/sec] (mean)
Time per request:       563.312 [ms] (mean)
Time per request:       2.817 [ms] (mean, across all concurrent requests)
T

In [ ]:
# Generic exception to prevent the continuation of executing other commands after opening the notebook.
raise Exception ("Stop right there!")

The command for "bringing down" all Docker services is:

In [136]:
!docker-compose down

[+] Running 0/0
 ⠋ Container pgadmin4_container  Stoppin...                                0.1s 
[+] Running 0/1
 ⠙ Container pgadmin4_container  Stoppin...                                0.2s 
[+] Running 0/1
 ⠹ Container pgadmin4_container  Stoppin...                                0.3s 
[+] Running 0/1
 ⠸ Container pgadmin4_container  Stoppin...                                0.4s 
[+] Running 0/1
 ⠼ Container pgadmin4_container  Stoppin...                                0.5s 
[+] Running 0/1
 ⠴ Container pgadmin4_container  Stoppin...                                0.6s 
[+] Running 0/1
 ⠦ Container pgadmin4_container  Stoppin...                                0.7s 
[+] Running 0/1
 ⠧ Container pgadmin4_container  Stoppin...                                0.8s 
[+] Running 0/1
 ⠇ Container pgadmin4_container  Stoppin...                                0.9s 
[+] Running 0/1
 ⠏ Container pgadmin4_container  Stoppin...                                1.0s 
[+] Running 0/1
 ⠋ Container p